<a href="https://colab.research.google.com/github/KjelleJ/enkla-ai-experiment/blob/main/AIX_5_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**OBJECT DETECTION**
Lokalisera föremål i en bild
---



In [ ]:
# Alla nödvändiga bibliotek
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle
from PIL import Image
import requests
import os

In [ ]:
# Hämta våra bilder

# hämta från url, lagra som lokal fil
def get_img_to_file(url, path):
  img_data = requests.get(url).content
  with open(path, 'wb') as handler:
      handler.write(img_data)

get_img_to_file("https://www.download.gubboit.se/hooks.JPG", "hooks")
get_img_to_file("https://www.download.gubboit.se/jakan.JPG", "jakan")

In [ ]:
plt.imshow(plt.imread('hooks'))

In [ ]:
plt.imshow(plt.imread('jakan'))

---
#VÅR MODELL
---

In [ ]:
# Läs in modellen
# Om fel "tf hub OSError: SavedModel file does not exist at" ta bort foldern och kör om...

# Ladda detektorn från TensorFlow Hub
detector = hub.load("https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1")

In [ ]:
# Funktionen object_detect använder vår modell för att göra 'object detection'
def object_detect(content_path, score_min):
    # Load content image
    content_image = plt.imread(content_path)
    # Convert to float32 numpy array, add batch dimension, and normalize to range [0, 1]. Example using numpy:
    # Det är samma krav på bilden som för style_transfer
    content_image = content_image.astype(np.float32)[np.newaxis, ...] / 255.

    # Detect
    # Problem med anropet därför signatures['default']
    output = detector.signatures['default'](tf.constant(content_image))

   # output är ett dictionary {}
    # vi är bara instresserade detection_scores, detection_class_entities och detection_boxes
    scores = output['detection_scores']
    classes = output['detection_class_entities']
    boxes = output['detection_boxes']
    # gör om från tensorer till numpy arrayer
    scores = scores.numpy()
    # gör oxå om från byte till string
    classes = classes.numpy().astype(str)
    boxes = boxes.numpy()

    for i in range(len(scores)):
        # skriv ut om score >= score_min
        if (scores[i] >= score_min):
            print(str(i), classes[i], scores[i])
            num_obj = i

    # plotta bilden med bounding boxes och objekt-nummer
    edge = ['r', 'y', 'b', 'g']
    image = Image.open(content_path)
    # Set the size of the plot figure
    plt.figure(figsize=(12, 12))
    plt.imshow(image)
    plt.axis('off')
    # Get the current reference
    ax = plt.gca()

    #for i in range(num_obj + 1):
    for i in range(len(scores)):
      if (scores[i] >= score_min):
        ymin = boxes[i][0]
        xmin = boxes[i][1]
        ymax = boxes[i][2]
        xmax = boxes[i][3]
        # Create a Rectangle patch
        width = image.width*(xmax - xmin)
        height = image.height*(ymax - ymin)
        rect = Rectangle((image.width*xmin, image.height*ymin), width, height,
                         linewidth=3, edgecolor=edge[i%4], facecolor='none')
        plt.text(image.width*xmin, image.height*ymin, str(i), color='red',
                 fontdict={"fontsize":20,"fontweight":'bold',"ha":"left", "va":"baseline"})
        # Add the patch to the Axes
        ax.add_patch(rect)
    plt.show()


---
#'OBJECT DETECTION' FÖR VÅRA BILDER
---

In [ ]:
# Objekt med sannolikhet större än 0.2
object_detect("jakan", 0.2)

In [ ]:
# Objekt med sannolikhet större än 0.1
object_detect("jakan", 0.15)

In [ ]:
object_detect("hooks", 0.2)

---
#PRÖVA MED EGEN BILD
---
OBS! Bara jpg-filer

In [ ]:
# Hitta en bild på nätet:
# Högerklicka på bilden, välj "kopiera bildadress",
# Dubbelklicka på URL i funkionsanropet, klistra in (ctrl-v)
get_img_to_file("URL", "minfil")

In [ ]:
object_detect("minfil", 0.2)

In [ ]:
# En fil på din PC
from google.colab import files

myfile = files.upload()

In [ ]:
# Ersätt ?????? med filnamn från cellen innan
object_detect("??????", 0.2)

---
#Prova på den här appen

Appen utför **Object Detection** på en video som visar rusningstrafik i NY filmat från en mc. Klicka på **Auto** så startar videon. Appen laddar först modellen från webbservern och det kan ibland ta lite tid.

https://gubboit.se/objectapp/